### GAAP Guides
https://www.fasb.org/projects/fasb-taxonomies/resources/taxonomy-implementation-guides


https://www.calcbench.com/element/ReceivablesNetCurrent

---

In [148]:
# Company("AAPL").get_filings(form="10-K").latest(10)

╭──────────────────────────────────────── Filings for Apple Inc. [320193] ────────────────────────────────────────╮
│                                                                                                                 │
│   #   Form       Description                                          Filing Date   Accession Number            │
│  ────────────────────────────────────────────────────────────────────────────────────────────────────────       │
│   0   10-K       Annual report for public companies                   2024-11-01    0000320193-24-000123        │
│   1   10-K       Annual report for public companies                   2023-11-03    0000320193-23-000106        │
│   2   10-K       Annual report for public companies                   2022-10-28    0000320193-22-000108        │
│   3   10-K       Annual report for public companies                   2021-10-29    0000320193-21-000105        │
│   4   10-K       Annual report for public companies                   

In [149]:
# tenk = Company("AAPL").get_filings(form="10-K").latest(1).obj()
# tenk.financials.xbrl_data.period_end

'2024-09-28'

In [13]:
from edgar import set_identity

set_identity("germankhivrenko@gmail.com")

In [14]:
import pandas as pd
from datetime import datetime
from edgar import Company


def get_tenk_df(ticker, n=10):
    lts_tenks = Company(ticker).get_filings(form="10-K").latest(n)
    
    tenk_dfs = []
    for tenk in lts_tenks:
        tenk_obj = tenk.obj()
        fins = tenk_obj.financials
        year = str(pd.to_datetime(fins.xbrl_data.period_end).year)
        
        tenk_dfs.append(
            pd.concat([
                fins.balance_sheet.get_dataframe()[["concept", year]],
                fins.income.get_dataframe()[["concept", year]],
                fins.cashflow.get_dataframe()[["concept", year]],
                fins.equity.get_dataframe()[["concept", year]],
                fins.comprehensive_income.get_dataframe()[["concept", year]],
            ])
        )
        tenk_dfs[-1].set_index("concept", inplace=True)
        # tenk_dfs[-1].drop_duplicates(inplace=True)
        tenk_dfs[-1] = tenk_dfs[-1][~tenk_dfs[-1].index.duplicated(keep=False)]
        # tenk_dfs[-1].reset_index(drop=True, inplace=True)
    # tenk_dfs.reverse()
    return pd.concat(tenk_dfs, axis=1)

In [15]:
df = get_tenk_df("UBER", 2)

In [40]:
df_1 = df.loc[["us-gaap_CashAndCashEquivalentsAtCarryingValue"]].reset_index(drop=True).astype(int)
df_2 = df.loc[["us-gaap_RestrictedCashAndCashEquivalentsAtCarryingValue"]].reset_index(drop=True).astype(int)
df_1

,2024,2023
0,5893000000,4680000000


In [39]:
df1 = pd.DataFrame({"A": [1], "B": [3]}, index=["apple"])
df2 = pd.DataFrame({"A": [10], "B": [30]}, index=["banana"])

df1.reset_index(drop=True) + df2.reset_index(drop=True)

,A,B
0,11,33


In [10]:
df.to_csv("uber_tenk_financials.csv")

In [203]:
df2 = get_tenk_df("UBER", 2)
df2

,2023,2024
concept,,
us-gaap_CashAndCashEquivalentsAtCarryingValue,4680000000,5893000000
us-gaap_MarketableSecuritiesCurrent,727000000,1084000000
us-gaap_RestrictedCashAndCashEquivalentsAtCarryingValue,805000000,545000000
us-gaap_AccountsReceivableNetCurrent,3404000000,3333000000
us-gaap_PrepaidExpenseAndOtherAssetsCurrent,1681000000,1390000000
...,...,...
uber_DepreciationAndAmortizationIncludingAmountFromOtherIncomeExpense,NaN,737000000
us-gaap_AccretionAmortizationOfDiscountsAndPremiumsInvestments,NaN,-251000000
us-gaap_PaymentsForRepurchaseOfCommonStock,NaN,-1252000000


In [204]:
list(df2.index)

['us-gaap_CashAndCashEquivalentsAtCarryingValue',
 'us-gaap_MarketableSecuritiesCurrent',
 'us-gaap_RestrictedCashAndCashEquivalentsAtCarryingValue',
 'us-gaap_AccountsReceivableNetCurrent',
 'us-gaap_PrepaidExpenseAndOtherAssetsCurrent',
 'us-gaap_AssetsCurrent',
 'us-gaap_RestrictedCashAndCashEquivalentsNoncurrent',
 'us-gaap_RestrictedInvestments',
 'uber_MarketableAndNonMarketableInvestments',
 'us-gaap_EquityMethodInvestments',
 'us-gaap_PropertyPlantAndEquipmentNet',
 'us-gaap_OperatingLeaseRightOfUseAsset',
 'us-gaap_IntangibleAssetsNetExcludingGoodwill',
 'us-gaap_Goodwill',
 'us-gaap_OtherAssetsNoncurrent',
 'us-gaap_Assets',
 'us-gaap_AccountsPayableCurrent',
 'uber_InsuranceReserveCurrent',
 'us-gaap_OperatingLeaseLiabilityCurrent',
 'us-gaap_AccruedLiabilitiesCurrent',
 'us-gaap_LiabilitiesCurrent',
 'uber_InsuranceReserveNoncurrent',
 'us-gaap_LongTermDebtNoncurrent',
 'us-gaap_OperatingLeaseLiabilityNoncurrent',
 'us-gaap_OtherLiabilitiesNoncurrent',
 'us-gaap_Liabilities

In [219]:
# df2.loc["us-gaap_CashAndCashEquivalentsAtCarryingValue"]

new_df = df2.loc[['us-gaap_CashAndCashEquivalentsAtCarryingValue']].copy()  # Use [['row1']] to keep it as a DataFrame
new_df.index = ['my_name']  # Change the index

new_df

,2023,2024
my_name,4680000000,5893000000


In [221]:
new_df2 = df2.loc[['us-gaap_AssetsCurrent']].copy()  # Use [['row1']] to keep it as a DataFrame
new_df2.index = ['my_name_2']  # Change the index

pd.concat([new_df, new_df2])

,2023,2024
my_name,4680000000,5893000000
my_name_2,11297000000,12245000000


In [222]:
new_df2.index

Index(['my_name_2'], dtype='object')

In [12]:
new_df2 - new_df2

NameError: name 'new_df2' is not defined

In [231]:
new_df2.loc["my_name_2"].astype(int) - new_df.loc["my_name"].astype(int)

2023    6617000000
2024    6352000000
dtype: int64

In [241]:
pd.concat([new_df2.loc["my_name_2"].astype(int).rename("Rev"), new_df.loc["my_name"].astype(int).rename("CostOfRev")], axis=1).T

,2023,2024
Rev,11297000000,12245000000
CostOfRev,4680000000,5893000000


In [236]:
new_df2.loc["my_name_2"].index

Index(['2023', '2024'], dtype='object')

In [ ]:
FIN_ITEM_MAP = {
    (): "revenue",  # Revenues, RevenueFromContractWithCustomerExcludingAssessedTax
    (): "cost_of_revenue",  # CostOfRevenue, gaap_CostOfGoodsAndServiceExcludingDepreciationDepletionAndAmortization
    (): "operating_expense",  # us-gaap_Revenues - us-gaap_CostOfRevenues - us-gaap_OperatingIncomeLoss
    (): "depreciation",  # gaap_DepreciationDepletionAndAmortization
    (): "provision_for_taxes",  # IncomeTaxExpenseBenefit
    (): "capital_expenditure",  # gaap_PaymentsToAcquirePropertyPlantAndEquipment
    (): "current_assets",  # us-gaap_AssetsCurrent
    (): "current_liabilities",  # s-gaap_LiabilitiesCurrent
}

In [ ]:
from abc import ABC, abstractmethod
from collections import Sequence


# OneToOneParser("revenue") - OneTwoOneParser("costofrevenue")
#
#


class FinancialsParser(ABC):
    @abstractmethod
    def parse(self, df: pd.DataFrame) -> pd.DataFrame:
        ...


class FinItemNotFound(Exception):
    pass


class OneToOneParser:
    def __init__(self, src: str, target: str) -> None:
        self._src = src
        self._target = target
    
    def parse(self, df: pd.DataFrame) -> pd.DataFrame:
        if self._src not in df.index:
            raise FinItemNotFound(f"Not found: {self._src}")
        return df.loc[[self._src]].copy().rename(index={self._src: self._target}).astype(int)  # FIXME: duplicate

    
class PriorityChainParser:
    def __init__(self, src_chain: Sequence[str], target: str) -> None:
        self._src_chain = src_chain
    
    def parse(self, df: pd.DataFrame) -> pd.DataFrame:
        for src in self.self._src_chain:
            if src in df.index:
                return df.loc[[src]].copy().rename(index={src: self._target}).astype(int)  # FIXME: duplicate
        raise FinItemNotFound(f"Not found any of: {self._src_chain}")
        
        
class CompoundParser:
    def __init__(self, parsers: dict[str, Sequence[FinancialsParser]]) -> None:
        self._parsers = parsers
        
    def parse(df: pd.DataFrame) -> pd.DataFrame:
        return pd.concat(parser.parse(df) for parser in self._parsers)